<a href="https://colab.research.google.com/github/JinHeeeKang/Playdata_Python/blob/master/0910.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


1. 데이터 로딩  
2. 하이퍼파라미터 설정하기  
3. 신경망 & 출력층 설계 및 구현    
    - 2층짜리 신경망
4. 미니배치 구성 & 학습     
5. 기울기 (gradient) 계산  
6. 가중치 업데이트   
7. 학습 경과 기록하기 (loss function)
    - 학습 결과가 안좋으면 거꾸로 돌아가보기


## 경사 하강법(수치미분)


In [ ]:
#1.데이터 로딩

import tensorflow as tf
mnist=tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test) = mnist.load_data()
x_train,x_test = x_train/225.0,x_test/255.0


In [ ]:
#2.필요한 함수 정의 sigmoid,등

def softmax(a):
    exp_a=np.exp(a)
    sum_exp_a=np.sum(exp_a)
    y=exp_a/sum_exp_a
    return y

def sigmoid(x):
    return 1/(1+np.exp(-x))

def cross_entropy_error(y,t):
    if y.ndim ==1:
        t=t.reshape(1,t.size)
        y=y.reshape(1,y.size)

    batch_size=y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [ ]:
#3.네트워크 클래스 정의

class TwoLayerNet:
    def __init__(self,input_size,hidden_size,output_size,weight_init_std=0.01):#가중치 초기화
        self.params={}
        self.params['W1']=weight_init_std*np.random.randn(input_size,hidden_size)
        self.params['W2']=weight_init_std*np.random.randn(hidden_size,output_size)

        self.params['b1']=np.zeros(hidden_size)
        self.params['b2']=np.zeros(output_size)

    #순전파
    def predict(self,x):
        W1,W2=self.params['W1'],self.params['W2']
        b1,b2=self.params['b1'],self.params['b2']

        a1=np.dot(x,W1)+b1
        z1=sigmoid(a1)

        a2=np.dot(z1,W2)+b2
        y=softmax(a2)

        return y

    #손실함수 x:입력데이터, t:정답데이터
    #x를 넣어 나온값과 t와의 차이
    def loss(self,x,t):
        y=self.predict(x)#self. :클래스 안에 있는 요함수를 써라
        loss=cross_entropy_error(y,t)
        return loss

    def accuracy(self,x,t):
        y=self.predict(x)
        y=np.argmax(y,axis=1)
        t=np.argmax(t,axis=1)
        accuracy =np.sum(y==t)/ float(x.shape[0])
        return accuracy
    def numerical_gradient(self,x,t):
        loss_W = lambda W:self.loss(x,t)
        grads={}
        grads['W1']=numerical_gradient(loss_W,self.params['W1'])
        grads['W2']=numerical_gradient(loss_W,self.params['W2'])
        grads['b1']=numerical_gradient(loss_W,self.params['b1'])
        grads['b2']=numerical_gradient(loss_W,self.params['b2'])
        return grads


In [ ]:
#4.학습 과정 수행

#4-1하이퍼파라미터 설정
iters_num=10000
train_size=x_train_flatten.shape[0]#train data의 전체사이즈
batch_size=100
learning_rate =0.1

#네트워크
network=TwoLayerNet(input_size=784,hidden_size=50,output_size=10)

train_loss_list=[]
#미니배치 구성 & 학습
for i in range(iters_num):
    #4-2배치데이터 획득
    #학습할때는 batch_size 만큼씩 뽑아서 사용
    batch_mask = np.random.choice(train_size,batch_size)
    x_batch = x_train_flatten[batch_mask]
    t_batch = y_train[batch_mask]#정답라벨

    #기울기 gradient 계산
    grad = network.numerical_gradient(x_batch,t_batch) 

    #4-3가중치 업데이트
    for key in ('W1','W2','b1','b2'):
        network.params[key] -= learning_rate *grad[key]
    
    #4-4학습결과 기록하기 loss function
    loss =network.loss(x_batch,t_batch)
    train_loss_list.append(loss)
    print(loss)


In [ ]:
x_batch

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
grad['b1'][0]

0.0002053552927705482

In [ ]:
#weight가 계산이 안되고 있는것 같음

 softmax 함수 부분에 있어서, input parameter값인 a는 1차원 배열을 가정하고 있는데, TwoLayerNet 클래스에서 학습은 배치 단위로 한번에 학습해서 a값이 2차원 배열을 가지고 있어서 모든 값들이 0에 가까운 값으로 return 되는 것이 아닐까요..?


## 오차역전파법
책 p. 167

In [ ]:
#사과,세금
#곱셈노드
class MulLayer:
    def __init__(self):
        self.x=None
        self.y=None

    def forward(self,x,y):
        self.x=x
        self.y=y

        out = x*y
        return out
    
    def backward(self,dout):
        dx=dout+self.x
        dy=dout+self.y

        return dx,dy

#덧셈노드
class AddLayer:
    def __init__(self):
        pass

    def forward(self,x,y):
        out = x+y
        return out
    
    def backward(self,dout):
        dx=dout*1
        dy=dout*1

        return dx,dy


In [ ]:
apple =100
apple_num=2
tax=1.1

#노드 정의
mul_apple_layer=MulLayer()
mul_tax_layer=MulLayer()

#forward
apple_price =mul_apple_layer.forward(apple,apple_num)
price=mul_tax_layer.forward(apple_price,tax)

#backward
dprice=1
dapple_price,dtax=mul_tax_layer.backward(dprice)
dapple,dapple_num=mul_apple_layer.backward(dapple_price)

In [ ]:
print(price)#너무 작은 값은 무시해도 됨
print(dapple_price,dtax)
print(dapple,dapple_num)

220.00000000000003
201 2.1
301 203


p.170~p.175 까지   
그림5-5


In [ ]:

class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        # 가중치와 편향 매개변수의 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        # 텐서 대응
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)  # 입력 데이터 모양 변경(텐서 대응)
        return dx


In [ ]:
#
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실함수
        self.y = None    # softmax의 출력
        self.t = None    # 정답 레이블(원-핫 인코딩 형태)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]#batch_size 증요ㅡ
        if self.t.size == self.y.size: # 정답 레이블이 원-핫 인코딩 형태일 때
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx


In [ ]:

class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

## 정리 X

In [ ]:
class TwoLayerNet:
    def __init__(self,input_size,hidden_size,output_size,weight_init_std=0.01):#가중치 초기화
        self.params={}
        self.params['W1']=weight_init_std*np.random.randn(input_size,hidden_size)
        self.params['W2']=weight_init_std*np.random.randn(hidden_size,output_size)

        self.params['b1']=np.zeros(hidden_size)
        self.params['b2']=np.zeros(output_size)

    #순전파
    def predict(self,x):
        W1,W2=self.params['W1'],self.params['W2']
        b1,b2=self.params['b1'],self.params['b2']

        a1=np.dot(x,W1)+b1
        z1=sigmoid(a1)

        a2=np.dot(z1,W2)+b2
        y=softmax(a2)

        return y

    #손실함수 x:입력데이터, t:정답데이터
    #x를 넣어 나온값과 t와의 차이
    def loss(self,x,t):
        y=self.predict(x)#self. :클래스 안에 있는 요함수를 써라
        loss=cross_entropy_error(y,t)
        return loss

    def accuracy(self,x,t):
        y=self.predict(x)
        y=np.argmax(y,axis=1)
        t=np.argmax(t,axis=1)
        accuracy =np.sum(y==t)/ float(x.shape[0])
        return accuracy
    def numerical_gradient(self,x,t):
        loss_W = lambda W:self.loss(x,t)
        grads={}
        grads['W1']=numerical_gradient(loss_W,self.params['W1'])
        grads['W2']=numerical_gradient(loss_W,self.params['W2'])
        grads['b1']=numerical_gradient(loss_W,self.params['b1'])
        grads['b2']=numerical_gradient(loss_W,self.params['b2'])
        return grads


In [ ]:
#기울기 계산
def numerical_gradient(f,x):
    h= 1e-4
    grad=np.zeros_like(x)

    for idx in range(x.shape[0]):
        tmp_val =x[idx]
        #y증가량 /x증가량 =기울기
        #y증가량 : f(x+h) -f(x-h)
        #f(x+h)
        x[idx] = tmp_val +h
        fxh1 = f(x)

        #f(x-h)
        x[idx] = tmp_val -h
        fxh2 = f(x)
        
        grad[idx]=(fxh1-fxh2)/(2*h)
    return grad

In [ ]:
import numpy as np

In [ ]:
def softmax(a):
    exp_a=np.exp(a)
    sum_exp_a=np.sum(exp_a)
    y=exp_a/sum_exp_a
    return y
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [ ]:
def cross_entropy_error(y,t):
    if y.ndim ==1:
        t=t.reshape(1,t.size)
        y=y.reshape(1,y.size)
    batch_size=y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [ ]:
import tensorflow as tf
mnist=tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test) = mnist.load_data()
x_train,x_test = x_train/225.0,x_test/255.0

In [ ]:
x_train_flatten=x_train.reshape([x_train.shape[0],-1])
x_test_flatten=x_train.reshape([x_test.shape[0],-1])

In [ ]:
# print(x_train.shape)
# print(x_train_flatten.shape)

In [ ]:
#하이퍼파라미터 설정
iters_num=10000
train_size=x_train_flatten.shape[0]
batch_size=100
learning_rate =0.1

#네트워크
network=TwoLayerNet(input_size=784,hidden_size=50,output_size=10)

train_loss_list=[]
#미니배치 구성 & 학습
for i in range(iters_num):
    #학습할때는 batch_size 만큼씩 뽑아서 사용
    batch_mask = np.random.choice(train_size,batch_size)
    x_batch = x_train_flatten[batch_mask]
    #정답라벨
    t_batch = y_train[batch_mask]

    #기울기 gradient 계산
    grad = network.numerical_gradient(x_batch,t_batch)

    #가중치 업데이트
    for key in ('W1','W2','b1','b2'):
        network.params[key] -= learning_rate *grad[key]
    
    #학습결과 기록하기 loss function
    loss =network.loss(x_batch,t_batch)
    train_loss_list.append(loss)
    print(loss)
